## Mosaics with 4D volumes

Mosaics display multiple image views simultaneously, often called “lightbox” mode. NiiVue lets you define both the orientation and position for any number of image snapshots. In this example, a slider is used to step through each 3D volume in a 4D time series, with slices chosen to be evenly spaced across the volume. Because the number of volumes and spatial extent are unknown until loading completes, these parameters are computed within the on_image_loaded() event, triggered once the image becomes available. This approach differs from typical sequential Python workflows, as load_volumes() is asynchronous—allowing images to load without freezing the user interface.

In [ ]:
import ipywidgets as widgets
from ipyniivue import NiiVue

nv = NiiVue()
nv.load_volumes([{"path":  "../images/mpld_asl.nii.gz"}])

def world_coord(i, j, k, affine):
    """Return world (x,y,z) for voxel index (i,j,k) using 4x4 affine list."""
    x = affine[0][0] * i + affine[0][1] * j + affine[0][2] * k + affine[0][3]
    y = affine[1][0] * i + affine[1][1] * j + affine[1][2] * k + affine[1][3]
    z = affine[2][0] * i + affine[2][1] * j + affine[2][2] * k + affine[2][3]
    return (x, y, z)

def axis_range(dims, affine):
    """
    Compute world-space min/max for each axis using the axis-midpoint method.

    - dims: list-like NIfTI dims (dims[1]=nx, dims[2]=ny, dims[3]=nz)
    - affine: 4x4 nested list
    - mid_mode: "between" uses dim/2.0 (geometric middle between voxel indices)
                "voxel"  uses (dim-1)/2.0 (center voxel index)
    Returns: dict with keys 'mn' and 'mx' mapping to [x_min,y_min,z_min] and [x_max,y_max,z_max]
    """
    nx = int(dims[1]) - 1
    ny = int(dims[2]) - 1
    nz = int(dims[3]) - 1
    # default: geometric center between voxel indices
    mid_x = nx / 2.0
    mid_y = ny / 2.0
    mid_z = nz / 2.0
    # X axis: vary i, hold j,k at mid
    i_lo = world_coord(0,    mid_y, mid_z, affine)
    i_hi = world_coord(nx, mid_y, mid_z, affine)
    # Y axis: vary j, hold i,k at mid
    j_lo = world_coord(mid_x, 0,    mid_z, affine)
    j_hi = world_coord(mid_x, ny, mid_z, affine)
    # Z axis: vary k, hold i,j at mid
    k_lo = world_coord(mid_x, mid_y, 0,    affine)
    k_hi = world_coord(mid_x, mid_y, nz, affine)
    # return range
    mn = [min(i_lo[0], i_hi[0]), min(j_lo[1], j_hi[1]), min(k_lo[2], k_hi[2])]
    mx = [max(i_lo[0], i_hi[0]), max(j_lo[1], j_hi[1]), max(k_lo[2], k_hi[2])]
    return mn, mx

@nv.on_image_loaded
def on_image_loaded(volume):
    """
    Event handler called when an image is loaded.

    Parameters
    ----------
    volume : ipyniivue.Volume
        The loaded image volume.
    """
    vol_slider4d.max = volume.n_frame_4d - 1
    mn, mx = axis_range(volume.hdr.dims, volume.hdr.affine)
    
    # attach to the volume instance
    volume.mnXYZ = mn
    volume.mxXYZ = mx
    update_mosaic()

# Start by showing every 5th slice, with 10 columns
#nv2.opts.slice_mosaic_string = full_mosaic(coords, ncols=init_cols, step=init_step)

axis_selector = widgets.Dropdown(
    options=["axial", "sagittal", "coronal"], description="View"
)
vol_slider4d = widgets.IntSlider(min=0, max=0, description="Volume")
column_slider = widgets.IntSlider(min=0, max=20, value=10, description="Columns")
row_slider = widgets.IntSlider(min=1, max=10, value=3, description="Rows")
size_slider = widgets.IntSlider(min=100, max=1000, value=300, description="Height")



def update_mosaic(*args):
    """Update the mosaic string."""
    prefix = axis_selector.value[0].upper()
    d = 0
    if prefix == 'A':
        d = 2
    elif prefix == 'C':
        d = 1
    mn_val = nv.volumes[0].mnXYZ[d]
    mx_val = nv.volumes[0].mxXYZ[d]
    cols = column_slider.value
    rows = row_slider.value
    N = cols * rows
    if N <= 0:
        nv.opts.slice_mosaic_string = prefix
        return

    # compute interval and positions such that first = mn + interval/2, last = mx - interval/2
    interval = (mx_val - mn_val) / float(N)
    positions = [mn_val + (i + 0.5) * interval for i in range(N)]

    # format with fixed decimals (3 here, change if you want)
    def fmt(v):
        return f"{v:.3f}"


    # build mosaic string: prefix + " " + numbers, inserting ";" after every `cols` items
    parts = [prefix]
    for i, pos in enumerate(positions):
        parts.append(fmt(pos))
        # after cols items (i is 0-based), insert ";" to start new row (except after final row if you prefer)
        if (i + 1) % cols == 0 and (i + 1) != N:
            parts.append(";")

    mosaic_string = " ".join(parts)
    nv.volumes[0].frame_4d = vol_slider4d.value
    nv.opts.slice_mosaic_string = mosaic_string

def update_height(*args):
    """Update widget height."""
    nv.height = size_slider.value

vol_slider4d.observe(update_mosaic, "value")
column_slider.observe(update_mosaic, "value")
row_slider.observe(update_mosaic, "value")
axis_selector.observe(update_mosaic, "value")
size_slider.observe(update_height, "value")

display(vol_slider4d)
display(column_slider)
display(row_slider)
display(size_slider)
display(axis_selector)
display(nv)